In [ ]:
# Image from Drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install --quiet -U --no-cache-dir gdown --pre
!pip install --quiet nlp-id
!pip install --quiet emoji
!pip install --quiet imblearn
!pip install --quiet gensim
!pip install --quiet shap
!pip install --quiet lime
!pip install -qq transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.9/7.9 MB 72.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.0/31.0 MB 16.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 81.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 240.9/240.9 KB 16.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 575.9/575.9 KB 17.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 275.7/275.7 KB 2.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 73.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.3/190.3 KB 24.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 74.5 MB/s eta 0:00:00


In [ ]:
from nlp_id.stopword import StopWord
from nlp_id.postag import PosTag
from nlp_id.lemmatizer import Lemmatizer

from imblearn.over_sampling import ADASYN 

from xgboost.sklearn import XGBClassifier

import emoji
import string
import re
import shap
import lime
import random
import numpy as np
import pandas as pd
import tensorflow as tf
import keras.backend as K

import nltk

from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords

nltk.download('punkt')

import seaborn as sns # untuk visualisasi
import matplotlib.pyplot as plt
plt.style.use('classic')
%matplotlib inline

import torch
import torch.nn.functional as F
import torch.nn as nn

from transformers import AutoTokenizer, AutoModel, get_constant_schedule_with_warmup
from transformers import TrainingArguments, Trainer, AutoConfig
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler

# potongan kode di bawah adalah agar gambar graph yang 
# dihasilkan mempunyai kualitas yang cukup baik
sns.set(rc={"figure.dpi":100, 'savefig.dpi':300})
sns.set_context('notebook')
sns.set_style("ticks")

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


# Ibrohim-Budi Data

In [ ]:
abusive_lexicon = pd.read_csv('/content/drive/MyDrive/Tugas Akhir/Data/abusive.csv')
dataset = pd.read_csv('/content/drive/MyDrive/Tugas Akhir/Data/re_dataset.csv', encoding='cp437')
kamus_alay = pd.read_csv('/content/drive/MyDrive/Tugas Akhir/Data/new_kamusalay.csv', encoding='cp437')

In [ ]:
# Tokenization & Emoticon Processor

regex_str = []
regex_str.append(r'<[^>]+>') # HTML tags
regex_str.append(r'(?:@[\w_]+)') # @-mentions
regex_str.append(r'(?:&[\w_]+)')
regex_str.append(r"(?:\#+[\w_]+[\w\'_\-]*[\w_]+)") # hash-tags
regex_str.append(r'http[s]?://(?:[a-z]|[0-9]|[$-_@.&amp;+]|[!*\(\),]|(?:%[0-9a-f][0-9a-f]))+') # URLs
regex_str.append(r'(?:(?:\d+,?)+(?:\.?\d+)?)') # numbers
regex_str.append(r"(?:[a-z][a-z'\-_]+[a-z])") # words with - and '
regex_str.append(r'(?:[\w_]+)') # other words
regex_str.append(r'(?:\S)') # anything else

tokens_re = re.compile(r'('+'|'.join(regex_str)+')', re.VERBOSE | re.IGNORECASE)

def tokenize(sentence):
    return [emoji.demojize(term) for term in tokens_re.findall(sentence)]

In [ ]:
# Stopwords Removal & Lemmatizing

lemmatizer = Lemmatizer()
stopword = StopWord()

punctuation = list(string.punctuation)
stops = stopword.get_stopword() + punctuation + ['rt', 'via', '…','•','“', 'user', 'url',
                                                  'xf0', 'x9f', 'x98', 'x84', 'xf0', 'x9f', 'x98', 'x84', 'xf0', 'x9f', 'x98', 'x84', 'xf', 'xe2', 'x80', 'x94', 'x82', 'xa4', 'n',
                                                  '1', '2', '3', '4', '5', '6', '7', '8', '9', '0']

def remove_stops_and_lemmatize(tokenized_sentence):
  processed_words = []
  for word in tokenized_sentence:
    if word.lower() not in stops:
      processed_words.append(lemmatizer.lemmatize(word))
  return processed_words

# BERT pytorch

In [ ]:
seed_val = 42
random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)

In [ ]:
if torch.cuda.is_available():
  device = torch.device("cuda")
  print("Mendukung")
  print("")
else:
  print("Tidak mendukung GPU hanya CPU")

Mendukung



In [ ]:
MAX_SEQ_LEN = 128
  # default max adalah 512, kalau kurang diganti padding
BATCH_SIZE = 2
  # besar batch setiap kali update parameter machine learning
WARMUP_PROPORTION = 0.4
MODEL_NAME = "indolem/indobert-base-uncased"
LEARNING_RATE = 5e-5
EPOCH = 30

In [ ]:
transformer = AutoModel.from_pretrained(MODEL_NAME)
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

Some weights of the model checkpoint at indolem/indobert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
# testing - input ids
text = dataset.iloc[0].Tweet
encoded_sent = tokenizer.encode(text,
                                add_special_tokens = True, 
                                max_length = MAX_SEQ_LEN, 
                                padding = "max_length", 
                                truncation = True 
                                )
print(encoded_sent)

[3, 17, 17446, 2014, 17257, 3519, 19154, 4082, 9875, 18, 5151, 929, 6849, 23840, 1493, 4082, 3798, 9875, 3774, 2523, 1500, 2393, 928, 18, 19723, 2393, 928, 17257, 5886, 928, 5, 5, 5, 11, 4, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


In [ ]:
# testing - tokenize
tokenizer.tokenize(text)

['-',
 'disaat',
 'semua',
 'cowok',
 'berusaha',
 'melacak',
 'perhatian',
 'gue',
 '.',
 'lo',
 '##e',
 'lantas',
 'remeh',
 '##kan',
 'perhatian',
 'yg',
 'gue',
 'kasih',
 'khusus',
 'ke',
 'el',
 '##o',
 '.',
 'basic',
 'el',
 '##o',
 'cowok',
 'beg',
 '##o',
 '!',
 '!',
 '!',
 "'"]

In [ ]:
# testing - dengan encode_plus
encoded_sent_2 = tokenizer.encode_plus(text,
                                add_special_tokens = True, 
                                max_length = MAX_SEQ_LEN, 
                                padding = "max_length", 
                                truncation = True 
                                )
print(encoded_sent_2['input_ids'])
print(encoded_sent_2['token_type_ids'])
print(encoded_sent_2['attention_mask'])

[3, 17, 17446, 2014, 17257, 3519, 19154, 4082, 9875, 18, 5151, 929, 6849, 23840, 1493, 4082, 3798, 9875, 3774, 2523, 1500, 2393, 928, 18, 19723, 2393, 928, 17257, 5886, 928, 5, 5, 5, 11, 4, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0

In [ ]:
att_mask = [int(token_id > 0) for token_id in encoded_sent]
print(att_mask)

[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


In [ ]:
# testing - decode
tokenizer.decode(encoded_sent)

"[CLS] - disaat semua cowok berusaha melacak perhatian gue. loe lantas remehkan perhatian yg gue kasih khusus ke elo. basic elo cowok bego!!!'[SEP] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD]"

In [ ]:
# test output dari transformer
# untuk mengetahui ada apa saja

output = transformer(torch.tensor([encoded_sent]))
print("Jenis: ", type(output))
print(output)

Jenis:  <class 'transformers.modeling_outputs.BaseModelOutputWithPoolingAndCrossAttentions'>
BaseModelOutputWithPoolingAndCrossAttentions(last_hidden_state=tensor([[[-0.7294, -0.1892, -0.2963,  ..., -1.3314, -0.0938,  0.3109],
         [-1.2393, -0.6297, -0.5218,  ...,  0.1562,  0.2129,  0.6951],
         [ 0.5921,  0.1207,  0.1834,  ..., -0.2508,  0.0808,  0.6528],
         ...,
         [ 0.0040,  0.2107, -0.6192,  ..., -0.4622,  0.1033,  0.4231],
         [ 1.2123,  0.8115, -1.1813,  ..., -1.0151,  0.3125, -0.5793],
         [ 1.5149, -0.0962, -0.6835,  ..., -1.0564,  0.8182, -0.8931]]],
       grad_fn=<NativeLayerNormBackward0>), pooler_output=tensor([[ 7.1824e-02, -2.7454e-02,  3.5052e-01,  4.7909e-01,  5.6877e-01,
         -5.8982e-01,  2.8015e-01,  2.7061e-01,  4.5224e-01,  6.9400e-01,
         -3.7959e-01,  1.7348e-01,  1.8875e-01,  1.8749e-02, -1.4743e-01,
         -5.1669e-01, -2.5885e-01,  4.4141e-01, -5.6202e-01,  1.7444e-01,
          2.9171e-01,  4.7255e-01,  1.6938e-01, 

In [ ]:
dataset.iloc[191]

Tweet            lebih baik jokowi mundur..
HS                                        1
Abusive                                   0
HS_Individual                             1
HS_Group                                  0
HS_Religion                               0
HS_Race                                   0
HS_Physical                               0
HS_Gender                                 0
HS_Other                                  1
HS_Weak                                   1
HS_Moderate                               0
HS_Strong                                 0
Name: 191, dtype: object

In [ ]:
docs = []

for i, row in dataset.iterrows():
  tweet = row.Tweet
  if row.HS == 0 and row.Abusive == 0:
    docs.append((tweet, "NHS/NA"))
  elif row.HS == 0 and row.Abusive == 1:
    docs.append((tweet, "NHS/A"))
  elif row.HS == 1 and row.Abusive == 0:
    docs.append((tweet, "HS/NA"))
  elif row.HS == 1 and row.Abusive == 1:
    docs.append((tweet, "HS/A"))

In [ ]:
len(docs)

13169

In [ ]:
# text classification dengan 2 label (multilabel)
random.shuffle(docs)

train_docs = docs[:70]
valid_docs = docs[70:]

label_map = {"HS/NA": [0, 0, 1, 0],
             "NHS/A": [0, 0, 0, 1],
             "HS/A": [0, 1, 0, 0],
             "NHS/NA": [1, 0, 0, 0],
            }

In [ ]:
train_docs

[("USER Najis anjer gua masih doyan lobang'", 'NHS/A'),
 ('IHSG dibuka menguat 17,22 poin atau 0,28% ke 6.302,98.', 'NHS/NA'),
 ('Media yang layak dibaca. Portal berita khusus bank-bank milik negara. Mengulas seputar ekonomi, bisnis, keuangan dan perbankan',
  'NHS/NA'),
 ("USER Yg anti cina komunis antek aseng kapitalis rakus zionis mana suaranya, lg enak2 ngentod ya'",
  'HS/A'),
 ('Babi... iya memang si Ahok gak jauh kayak Babi..', 'HS/A'),
 ("Bikin coldbrew agak ngasal. Pagi diminum cuman demi melek instan aja. Tai lah gua masih ga ngerti kopi enak apa ga kayak apa. Hahahaahah'",
  'NHS/A'),
 ("USER USER USER USER USER USER USER USER USER Kasarnya mau komunis free sex sekalipun, klo punya cadangan minyak melimpah, mudah mencapai HDI tinggi.'",
  'NHS/NA'),
 ('USER kenapa ulama tidak bereaksi terhadap USER &gt; ini lebih gila menghina Islam. Apakah cuma senilai dunia agama kalian.',
  'HS/NA'),
 ('Apa dampak dari lengsernya rezim adalah restar ekonomi dan akan banyak yang bangkrut d

In [ ]:
def create_data_loader(instances, label_map = label_map):
  input_ids = []
  input_att_mask = []
  input_label = []
  for (instance, label) in instances:
    encoded_sent = tokenizer.encode(instance, \
                                  add_special_tokens = True, \
                                  max_length = MAX_SEQ_LEN, \
                                  padding = "max_length", \
                                  truncation = True)
    input_ids.append(encoded_sent)
    input_att_mask.append([int(token_id > 0) for token_id in encoded_sent])
    input_label.append(label_map[label])
  
  # ubah ke Torch's Tensor
  input_ids = torch.tensor(input_ids)
  input_att_mask = torch.tensor(input_att_mask)
  input_label = torch.tensor(input_label)

  # gunakan Torch's Dataset
  dataset = TensorDataset(input_ids, input_att_mask, input_label)

  # mengembalikan sebuah abstraksi DataLoader dari Torch
  return DataLoader(dataset,
                    sampler = SequentialSampler(dataset), 
                    #sampler = RandomSampler(dataset), 
                    batch_size = BATCH_SIZE)

In [ ]:
train_instances = create_data_loader(train_docs)
valid_instances = create_data_loader(valid_docs)

In [ ]:
train_instances

In [ ]:
# Classifier adalah layer on top of BERT yang digunakan untuk klasifikasi
# inputnya adalah vektor CLS dari BERT
class Classifier(nn.Module):
    def __init__(self, input_size = 768, output_size = 4, dropout_rate = 0.1):
        super(Classifier, self).__init__()
        layers = []
        layers.append(nn.Linear(input_size, output_size))
        layers.append(nn.LeakyReLU(0.2, inplace = True))
        layers.append(nn.Dropout(dropout_rate))
        layers.append(nn.Softmax(dim = -1))
        self.layers = nn.Sequential(*layers)

    def forward(self, input):
        return self.layers(input)


In [ ]:
# jika Anda ingin mengetahui ukuran hidden_size di setiap
# timestep pada BERT / Encoder-nya Transformer
config = AutoConfig.from_pretrained(MODEL_NAME)
hidden_size = int(config.hidden_size)
print(hidden_size)

768


In [ ]:
# instansiasi classifier
classifier = Classifier(input_size = config.hidden_size)

In [ ]:
# test output dari classifier
bert_cls = transformer(torch.tensor([encoded_sent])).pooler_output
print(classifier(bert_cls))

tensor([[0.2276, 0.2268, 0.3104, 0.2352]], grad_fn=<SoftmaxBackward0>)


In [ ]:
# jika GPU available
# Put everything in the GPU if available
if torch.cuda.is_available():    
  classifier.cuda()
  transformer.cuda()

In [ ]:
# models parameters
transformer_vars = [p for p in transformer.parameters()]
classifier_vars = [p for p in classifier.parameters()]
all_vars = transformer_vars + classifier_vars

In [ ]:
# ada berapa parameter di classifier
# secara teori, ada 768 x 3 + 3 bias
total_vars = 0
for v in classifier_vars:
  print(v)
  total_vars += len(torch.flatten(v))
print("Total parameters di Classifier: ", total_vars)

Parameter containing:
tensor([[ 0.0242, -0.0289, -0.0116,  ..., -0.0355, -0.0062,  0.0205],
        [ 0.0183, -0.0198,  0.0151,  ...,  0.0236,  0.0231,  0.0146],
        [-0.0227,  0.0002,  0.0085,  ..., -0.0286, -0.0233,  0.0164],
        [-0.0328,  0.0256, -0.0007,  ..., -0.0197, -0.0297, -0.0161]],
       device='cuda:0', requires_grad=True)
Parameter containing:
tensor([ 0.0089, -0.0039, -0.0178, -0.0046], device='cuda:0',
       requires_grad=True)
Total parameters di Classifier:  3076


In [ ]:
# training

# optimizer
optimizer = torch.optim.AdamW(all_vars, lr = LEARNING_RATE)

# kita gunakan scheduler untuk warming-up
# Vaswani mengusulkan suatu rumus penentuan learning_rate saat warming-up
# dan setelahnya
num_train_instances = len(train_instances)
num_train_steps = int(num_train_instances / BATCH_SIZE * EPOCH)
num_warmup_steps = int(num_train_steps * WARMUP_PROPORTION)

#scheduler = get_constant_schedule_with_warmup(optimizer, 
#                                              num_warmup_steps = num_warmup_steps)

for epoch_i in range(0, EPOCH):
  print(f"Epoch {epoch_i + 1} / {EPOCH}")
  # Reset the total loss for this epoch.
  tr_loss = 0

  # Put the model into training mode.
  transformer.train() 
  classifier.train()

  # Untuk setiap batch di training data
  for step, batch in enumerate(train_instances):
    print(f">>>> Batch {step + 1}")

    # Unpack sebuah batch dari DataLoader
    batch_input_ids = batch[0].to(device)
    batch_input_att_mask = batch[1].to(device)
    batch_label = batch[2].to(device)

    # Encode real data in the Transformer
    bert_outputs = transformer(batch_input_ids, attention_mask = batch_input_att_mask)
    cls_hidden_states = bert_outputs.pooler_output
    probs = classifier(cls_hidden_states)

    # Categorical Cross-Entropy
      # + (1 - batch_label) * (1 - torch.log(probs))
    loss = -torch.mean(torch.sum(batch_label * torch.log(probs), dim = 1))

    # Agar gradient tidak menumpuk; ini diperlukan sebelum
    # menghitung gradient dengan loss.backward()
    optimizer.zero_grad()

    # hitung gradient
    loss.backward()

    # update parameter
    optimizer.step()

    # akumulasi loss dalam 1 epoch
    tr_loss += loss.item()

    # update learning rate
    #scheduler.step()

  # rata-rata loss dalam 1 epoch
  avg_loss = tr_loss / len(train_instances)

  print(f"Average loss: {avg_loss}")

  ### Uji coba di Validation Data
  print("Uji coba di Validation Data ...")

  # model eval, agar layer seperti dropout yang menghasilkan
  # sesuai yang random tidak digunakan
  classifier.eval()
  transformer.eval()

  val_loss = 0
  pred_labels = []
  true_labels = []

  # Untuk setiap batch di validation data
  for step, batch in enumerate(valid_instances):

    # Unpack sebuah batch dari DataLoader
    batch_input_ids = batch[0].to(device)
    batch_input_att_mask = batch[1].to(device)
    batch_label = batch[2].to(device)

    # jangan track gradient! ini sedang evaluasi, bukan training
    with torch.no_grad(): 
      bert_outputs = transformer(batch_input_ids, attention_mask = batch_input_att_mask)
      cls_hidden_states = bert_outputs.pooler_output
      probs = classifier(cls_hidden_states)

      # Categorical Cross-Entropy
      loss = -torch.mean(torch.sum(batch_label * torch.log(probs), dim = 1))

      # akumulasi loss di validation set
      val_loss += loss.item()

    # akumulasi prediksi
    _, pred_l = torch.max(probs, dim = 1) # outnya adalah 2-tuple: (max, max_indices)
    pred_labels += pred_l.detach().cpu()
    _, true_l = torch.max(batch_label, dim = 1)
    true_labels += true_l.detach().cpu()

  # average loss
  avg_val_loss = val_loss / len(valid_instances)

  # accuracy
  pred_labels = torch.stack(pred_labels).numpy()
  true_labels = torch.stack(true_labels).numpy()
  val_accuracy = np.sum(pred_labels == true_labels) / len(pred_labels)
  print(f"Average loss di validation data: {avg_val_loss}")
  print(f"Accuracy di validation data: {val_accuracy}")

  print("")

Epoch 1 / 30
>>>> Batch 1
>>>> Batch 2
>>>> Batch 3
>>>> Batch 4
>>>> Batch 5
>>>> Batch 6
>>>> Batch 7
>>>> Batch 8
>>>> Batch 9
>>>> Batch 10
>>>> Batch 11
>>>> Batch 12
>>>> Batch 13
>>>> Batch 14
>>>> Batch 15
>>>> Batch 16
>>>> Batch 17
>>>> Batch 18
>>>> Batch 19
>>>> Batch 20
>>>> Batch 21
>>>> Batch 22
>>>> Batch 23
>>>> Batch 24
>>>> Batch 25
>>>> Batch 26
>>>> Batch 27
>>>> Batch 28
>>>> Batch 29
>>>> Batch 30
>>>> Batch 31
>>>> Batch 32
>>>> Batch 33
>>>> Batch 34
>>>> Batch 35
Average loss: 1.242272557956832
Uji coba di Validation Data ...
Average loss di validation data: 1.3864603311869934
Accuracy di validation data: 0.3483472020764944

Epoch 2 / 30
>>>> Batch 1
>>>> Batch 2
>>>> Batch 3
>>>> Batch 4
>>>> Batch 5
>>>> Batch 6
>>>> Batch 7
>>>> Batch 8
>>>> Batch 9
>>>> Batch 10
>>>> Batch 11
>>>> Batch 12
>>>> Batch 13
>>>> Batch 14
>>>> Batch 15
>>>> Batch 16
>>>> Batch 17
>>>> Batch 18
>>>> Batch 19
>>>> Batch 20
>>>> Batch 21
>>>> Batch 22
>>>> Batch 23
>>>> Batch 24
>